In [85]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple,trunc_normal_
import torch.nn.functional as F

In [96]:
class Args():
    ...
args = Args()

args.x = args.img = torch.randn((1,3,12,12))
print("img -> b,inc,h,w:",list(args.img.shape))

img -> b,inc,h,w: [1, 3, 12, 12]


In [97]:
# patch_embed  from b,inc,h,w -> b,num_patches,embed_dim
args.embed_dim = 6 # default: 96
args.patch_size = 4 # default: 4
args.patch_resolution = [args.img.shape[2] // args.patch_size , args.img.shape[3] // args.patch_size] 
args.num_patches = args.patch_resolution[0] * args.patch_resolution[1]
args.norm_layer = nn.LayerNorm # default = nn.LayerNorm . 
if args.norm_layer is None:
    args.norm_layer = nn.Sequential() 
def patch_embed(args):
    x = nn.Conv2d(in_channels=args.img.shape[1], 
                out_channels= args.embed_dim,
                kernel_size=args.patch_size,
                stride=args.patch_size)(args.img)
    x = torch.flatten(x,start_dim=2).transpose(1,2)
    x = args.norm_layer(x.shape[1:])(x) # 在通道上做归一化,即在num_patches维度上做归一化
    return x
args.x = args.patch_embed = patch_embed(args)

assert args.num_patches == args.patch_embed.shape[1]
print("patch_embed -> b,num_patches,embed_dim:",list(args.patch_embed.shape))

patch_embed -> b,num_patches,embed_dim: [1, 9, 6]


In [100]:
# absolute pos embed
args.ape = False # dafault = False
def abs_pos_embed(args):
    return trunc_normal_(nn.Parameter(torch.zeros(1,args.num_patches,args.embed_dim)),std=.02)
    
if args.ape:
    args.x = args.x + abs_pos_embed(args)

In [101]:
# pos drop
args.drap_rate = 0. # defatult 0
args.x = args.pos_drop = nn.Dropout(p=args.drap_rate)(args.x)

AttributeError: 'Args' object has no attribute 'pos_drop_rate'

In [107]:

class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - \
            coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(
            1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - \
            1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index",
                             relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C //
                                  self.num_heads).permute(2, 0, 3, 1, 4)
        # make torchscript happy (cannot use tensor as tuple)
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(
            2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N,
                             N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def extra_repr(self) -> str:
        return f'dim={self.dim}, window_size={self.window_size}, num_heads={self.num_heads}'


def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size,
               W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous(
    ).view(-1, window_size, window_size, C)
    return windows



def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image

    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size,
                     window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x

def drop_path(x, drop_prob: float = 0., training: bool = False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).

    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.

    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output

In [106]:
# Model
args.depths = [1,1,2,1] # default = [2,2,6,2]
args.num_heads = [1,1,1,1,] # defautl = [3,6,12,24]
args.window_size = 7 # default = 7
args.mlp_ratio = 4. # default = 4.
args.qkv_bias = True # default = True
args.qkv_scale = None # default = None
args.attn_drop_rate = 0. # default = 0.
args.drop_path_rate = 0.1 # default = 0.1
args.patch_norm = True # default = True
args.use_checkpoint = False # default = False
dpr = [x.item() for x in torch.linspace(0,args.drop_path_rate,sum(args.depths))] # stochastic depth decay relu

In [ ]:

for i_layer in range(len(args.depths)):
    """one BasicLayer""""
    dim = int(args.embed_dim * 2 ** i_layer)
    input_resolution = (args.patch_resolution[0] // (2**i_layer),args.patch_resolution[1] // (2**i_layer))
    depth = args.depths[i_layer]
    num_heads = args.num_heads[i_layer]
    drop_path = dpr[sum(args.depths[:i_layer]):sum(args.depths[:i_layer+1])]
    downsample = True if (i_layer < len(args.depths) -1 ) else False

    for i in range(depth):
        """SwinTransformerBlock"""
        shift_size = 0 if (i%2==0) else args.window_size // 2
        drop_path_i = drop_path[i] if isinstance(drop_path,list) else drop_path
        if min(args.input_resolution) <= args.window_size:
            shift_size = 0 # don't partition windows
            args.window_sise = min(args.input_resolution)
        assert 0 <= shift_size <= args.window_size,"shift_size must in 0-window_size"

        H,W = input_resolution
        B,L,C = args.x.shape
        assert L == H*W, "input feature has wrong size"
        shorcut = x = args.x
        x = args.norm_layer(x.shape[1])(x).view(B,H,W,C)

        """W-WSA/SW-WSA"""

        # cyclic shift
        shift_x = x if shift_size == 0 else torch.roll(x,shifts=(-shift_size,-shift_size),dims=(1,2))

        # partition windows -> (num_windows*B, window_size, window_size, C) -> (num_windows*B, window_size*window_size, C)
        x_windows = window_partition(shift_x,args.window_size).view(-1,args.window_size*args.window_size,C)
        # attn mask
        if shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = input_resolution  
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -args.window_size),
                        slice(-args.window_size, -shift_size),
                        slice(-shift_size, None))
            w_slices = (slice(0, -args.window_size),
                        slice(-args.window_size, -shift_size),
                        slice(-shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            # nW, window_size, window_size, 1
            mask_windows = window_partition(img_mask, self.window_size)
            mask_windows = mask_windows.view(-1,
                                             self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(
                attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None
        # W-MSA/SW-MSA
        # nW*B, window_size*window_size, C
        attn_windows = WindowAttention(
            dim,window_size=(args.window_size,args.window_size),num_heads=args.num_heads,
            qkv_bias=args.qkv_bias,qk_scale=args.qkv_scale,attn_drop=args.attn_drop_rate,proj_drop=args.drop_path)(x_windows,mask = attn_mask)

        # merge windows 
        attn_windows = attn_windows.view(-1,args.window_size,args.window_size,C)
        shifted_x = window_reverse(attn_windows,args.window_size,H,W) # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(shift_size,shift_size), dim=(1,2))
        else:
            x = shifted_x
        x = x.view(B,H*W,C)

        # FFN

        x = shortcut + drop_path(x,drop_path_i,training=True)
        mlp_x = nn.Sequential(
            nn.Linear(dim,int(dim*args.mlp_ratio)),
            nn.GELU()
            nn.Linear(int(dim*args.mlp_ratio),dim),
            nn.GELU(),
            nn.Dropout(args.drop_rate)
        )(args.norm_layer(x))
        x = x + drop_path(mlp_x)

    if args.downsample:
        x = donwsample(x)
